In [ ]:
import datasets
from src import data_loading

dataset = datasets.load_dataset("lmarena-ai/arena-human-preference-140k")
data = data_loading.load_training_data(dataset["train"].to_pandas())
print(f"Successfully loaded {len(data.entries)} entries")

In [ ]:
from src.models.dn_embedding_model import DnEmbeddingModel
from src.models.optimizers.adamw_spec import AdamWSpec
from src.utils.data_split import ValidationSplit


model = DnEmbeddingModel(
    hidden_dims=[512, 512, 256, 256, 128, 128, 64, 64],
    optimizer_spec=AdamWSpec(
        learning_rate=5e-5,
        lr_decay_gamma=0.998,
    ),
    balance_model_samples=True,
    
    embedding_model_epochs=50,
    embedding_model_name="all-mpnet-base-v2",
    embedding_model_hidden_dims=[512, 512, 256, 256, 128, 128, 64, 64],
    embedding_model_optimizer_spec=AdamWSpec(
        learning_rate=5e-5,
        lr_decay_gamma=0.998,
    ),
    triplet_margin=0.2,
    regularization_weight=1e-5,
    min_model_comparisons=20,
    identity_positive_ratio=0.9,
    seed=42,
    print_every=1,
)

model.train(data, validation_split=ValidationSplit(val_fraction=0.2, seed=42), epochs=120, batch_size=512)
model.save("first-try-embedding-model-with-all-mpnet-base-v2")

In [ ]:
model.last_timer.inspect(1)
print()
model.embedding_model.last_timer.inspect(1)

In [ ]:
import matplotlib.pyplot as plt

from src.plotting_utils import plot_loss, plot_accuracy

history = model.embedding_model._epoch_logs
_, axes = plt.subplots(3, 2, figsize=(15, 15))

plot_loss(axes[0, 0], [e.train_loss for e in history], "Training loss")
plot_loss(axes[0, 1], [e.val_loss for e in history], "Validation loss")
plot_accuracy(axes[1, 0], [e.train_triplet_accuracy for e in history], "Training accuracy")
plot_accuracy(axes[1, 1], [e.val_triplet_accuracy for e in history], "Validation accuracy")
plot_loss(axes[2, 0], [e.train_reg_loss for e in history], "Training regularization loss")
axes[2, 1].plot([e.duration for e in history])
axes[2, 1].set_xlabel("Epoch")
axes[2, 1].set_ylabel("Duration (s)")

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from src.plotting_utils import plot_loss, plot_accuracy

history = model.get_history()
_, axes = plt.subplots(2, 2, figsize=(15, 10))

plot_loss(axes[0, 0], history.total_loss, "Training loss")
plot_loss(axes[0, 1], history.val_loss, "Validation loss")
plot_accuracy(axes[1, 0], history.train_accuracy, "Training accuracy")
plot_accuracy(axes[1, 1], history.val_accuracy, "Validation accuracy")

plt.tight_layout()
plt.show()

In [ ]:
from src.data_models.data_models import InputData
from src.preprocessing.prompt_embedding_preprocessor import PromptEmbeddingPreprocessor
from src.utils import data_split

_, validation_prompts = data_split.train_val_split(PromptEmbeddingPreprocessor.filter_out(data), val_fraction=0.2, seed=42)
results = model.predict(InputData(
        prompts=validation_prompts,
        model_names=model._model_encoder.names,
    ),
    batch_size=512,
).scores

sorted_results = sorted(
    [(name, float(np.mean(scores)), float(np.std(scores))) for name, scores in results.items()],
    key=lambda x: x[1],
    reverse=True
)

for name, mean_score, std_score in sorted_results:
    print(f"{(name+":"):<{max(len(name) for name, _, _ in sorted_results)+1}s} {mean_score:.6f} ± {std_score:.6f}")